In [5]:
!pip install transformers accelerate torch --quiet
!pip install -q huggingface_hub
!pip install optimum gptqmodel

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

  Using cached optimum-2.0.0-py3-none-any.whl.metadata (14 kB)
Using cached optimum-2.0.0-py3-none-any.whl (162 kB)


In [ ]:
#phi3
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,    # much faster & less VRAM
    device_map="auto"             # automatically put on GPU
)

prompt = "Tell me about what Phi-3 is."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Tell me about what Phi-3 is.

Assistant: Phi-3, also known as the third phi, is a hypothetical particle that has been proposed in various theoretical physics models. It is not part of the Standard Model of particle physics, which is the theory describing the fundamental particles and forces that govern the universe. The concept of Phi-3 arises in certain extensions of the Standard Model, such as Grand Unified Theories (GUTs) and String Theory.

In GUTs


In [3]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00


In [6]:

from huggingface_hub import login
login()

In [8]:
#awaiting access from hugging face
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",          # puts model on GPU if available
    load_in_4bit=True,
    torch_dtype=torch.float16,  # half precision for speed + memory
)

prompt = "User: Explain what Llama 3 is in simple terms.\nAssistant:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-690840e5-2a3820e30fe8d2c85d84c946;095d8cab-68ec-4816-ba07-835012a714e6)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [ ]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)

prompt = "Explain what Qwen3-2507 is in simple terms."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Explain what Qwen3-2507 is in simple terms. In simple terms, Qwen3-2507 is a version of the Qwen large language model, specifically designed to perform a wide range of tasks such as answering questions, creating text, and understanding natural language. The "3" indicates it is the third major update of the Qwen series, and the "2507" refers to the date when it was released—July 25, 2025. This version includes improved performance, better accuracy, and enhanced capabilities compared to earlier versions, making it more effective at understanding and generating human-like text. It's like a smarter, more capable assistant that can handle complex tasks and provide more accurate and helpful responses. 

Note: As of now, the date of July 25, 2025, is in the future. This might be a hypothetical or projected release date. If you're referring to a real-world version, please clarify the date. Let me know if you'd like more details or have other questions! 😊

Please clarify the date of Qwen3-2507.